In [2]:
import time, os, sys
import sagemaker, boto3

In [3]:
sess = boto3.Session()
sm   = sess.client('sagemaker')
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)

In [4]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent

training_experiment = Experiment.create(
                                experiment_name = "sagemaker-ml-experiment3", 
                                description     = "segmenting brain tumor from images", 
                                sagemaker_boto_client=sm)


In [5]:
single_gpu_trial = Trial.create(
    trial_name = 'sagemaker-single-gpu-training3', 
    experiment_name = training_experiment.experiment_name,
    sagemaker_boto_client = sm,
)

trial_comp_name = 'single-gpu-training-job'
experiment_config = {"ExperimentName": training_experiment.experiment_name, 
                       "TrialName": single_gpu_trial.trial_name,
                       "TrialComponentDisplayName": trial_comp_name}

In [96]:
from sagemaker.tensorflow import TensorFlow

# hyperparams={'epochs'       : 30,
#              'learning-rate': 0.01,
#              'batch-size'   : 256,
#              'weight-decay' : 2e-4,
#              'momentum'     : 0.9,
#              'optimizer'    : 'adam'}

bucket_name = 'sagemaker-hi-data'
output_path = f's3://{bucket_name}/jobs'
metric_definitions = [{'Name': 'val_acc', 'Regex': 'val_acc: ([0-9\\.]+)'}]

tf_estimator = TensorFlow(entry_point          = 'brain_tumor_seg_project1.py', 
                          output_path          = f'{output_path}/',
                          code_location        = output_path,
                          role                 = role,
                          train_instance_count = 1, 
                          train_instance_type  = 'ml.m4.xlarge',
                          framework_version    = '2.3', 
                          py_version           = 'py37',
                          script_mode          = True,
                          metric_definitions   = metric_definitions,
                          sagemaker_session    = sagemaker_session)

job_name=f'tensorflow-single-gpu-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'

tf_estimator.fit("s3://sagemaker-hi-data/ml_project1_data/", 
                job_name = job_name,
                experiment_config=experiment_config)

# tf_estimator.fit({'training'  : datasets,
#                   'validation': datasets,
#                   'eval'      : datasets},
#                  job_name = job_name,
#                  experiment_config=experiment_config)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: tensorflow-single-gpu-2023-07-01-15-38-58


2023-07-01 15:38:59 Starting - Starting the training job...
2023-07-01 15:39:24 Starting - Preparing the instances for trainingProfilerReport-1688225939: InProgress
.........
2023-07-01 15:40:47 Downloading - Downloading input data...
2023-07-01 15:41:27 Training - Downloading the training image...
2023-07-01 15:41:48 Training - Training image download completed. Training in progress...2023-07-01 15:42:04.977910: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-07-01 15:42:04.978106: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-07-01 15:42:05.016365: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-07-01 15:42:06,533 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2023-07-01 15:42:06